<a href="https://www.kaggle.com/code/kevinsimorangkir21/sentiment-analysis-using-gru?scriptVersionId=207273591" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Sentiment Analysis Using GRU

